## APRIL 25

### cans

- paper comparing only baseline emotional with baseline weight - bigger sample size than in longitudinal
- also, linking baseline emotional with dropping out, and whole program completion (percentage of completers, weight loss at completion)
- use frequency of measurements and number of visits as indicators of adherence
- 10% WL vs dropout as competing risks (events that prevent the outcome of interest) in a survival model (Fine-Gray model)
- 

### musts

- rowcleaned measurements has to go into the first paper sql pipeline, duplicates have to be removed 
    - this can be done in the subsetted SQL too. Dilemma: universally good to have row-cleaning, duplicate and outlier removal universally. But, identifying duplicates and outliers in the data frames other than measurements is an effort, and time loss as of mid april 2025. (*i know this is not the right attitude, but this is the leaner one*)
- when checking if a baseline/final measurement is within the range of a medical record, don't look whether it is within/without the start and end dates. Rather, look if it is within a 10-15-day window of both dates. Sometimes the first measurement is months after the start of a prescription, and that can't be taken as a reliable baseline value. Identifying cases like this can also help spotting loopholes in the analysis pipeline. 
- correcting for GDPR
- when using genetics and considering personalized recommendations, correcting for dates


### 16/4/25

[15.30] continuing

[11.10 **problems for future self:**]
- revise and comment the modules that complete medical records, and create the survival analysis input
- hunt down the issues noted at 9.05 

[9.05] issues: 
- medical records complete is smaller than original medical records, look into it - *SOLVED - data is lost because any medical record with no measurement associated to it is dropped*
- dropout handling: only those with 0 days of followup are considered dropouts, while dropout should be someone who does not achieve nor success nor target measurement time SHOULDVE SPECIFIED DF IN QUESTION
- 14% WL as True for 15% SHOULDVE SPECIFIED DF IN QUESTION - I GUESS SA INPUT
- measurements_with_metadata probably has irrelevant columns at the end

[8.05] commit message: 

Updated the way that the Paper 1-specific SQL is created. After subsetting the medical_records table, all other tables were subsetted by patient_id only. Prescriptions however contained medical records from some patients that were lacking emotional data. 

The updated code is supposed to filter prescriptions by both patient and medical record IDs, really only keeping records with complete emotional data availability. 


number of prescriptions when using the old code, that filters prescriptions by patient ID only: 16937
number when using the new code that filters both by patient and medical record IDs: 10190

started at 7.40 today


### 15/4/25

[**problems for future self:**] 18% of medical record-derived data in the lates SA input tables is missing. That means that the medical records for those patients are not available, and that there is no consistency regarding the records contained in the study-specific SQL. That probably makes me have to look several steps back - in the future! [*problem solved on 16/4/2025 - the SQL subsetting was done incorrectly, prescriptions were filtered by patient ID and not medical record ID, and so in the step of linking prescriptions to measurements, many measurements that had no corresponding emotional values were retained*]

[19.46] yay, survival analysis input done! But there are issues - it all needs to be revised, all the pipeline for this paper-specific analysis

[16.30] prompt to get survival analysis input

Write a module that pulls data from measurements_with_metadata, and obtains input data organized for survival analysis. The outcome of interest is 10% weight loss. 

The code should group the measurements (rows) by patient id and medical record ID (measurements from the same treatment of the same patient), and take the first measurement of each group as the reference baseline measurement. 
Counting from the first measurement, it should look at the following measurements in a range of 60, +/- 10 days, 
and check the porcentual amount of weight lost in each timepoint. 
If a weight loss over 10% is achieved, the days it took to achieve it should be noted. 
If the patient did not follow the treatment for at least 60 +/- 10 days, the last measurement should be noted.

The output should be a data frame called sa_60d_10%. It should have the following columns: 
- patient_id - text, patient ID
- medical_record_id - text, medical record ID
- first_measurement_date - datetime, the first measurement in the group
- end_measurement_date - datetime, either the last measurement in the group (in case of dropout before 60 days, OR not achieving 10% weight loss in 60 days) OR the first measurement where 10% weight loss is achieved
- days_to_end_measurement - numeric, days passed between the first and last measurement date
- nr_measurements - numeric, the number of measurements between the first and the end measurement
- baseline_weight - numeric, the weight at time of the first measurement
- final_weight - numeric, the weight at the time of the last measurement
- wl% - numeric, the porcentual difference between the baseline and final weight values
- 10%_achieved - bool, yes/no, yes if final weight is over 10% less than baseline, no if not
- dropout - bool, yes/no, yes if didn't reach 60+/- 10 days of followup AND didn't reach over 10% weight loss








[13.15] **reu Alfredo** 
- 6 regresiones para 6 variables, y luego uno para todo
- pérdida de peso a los 50, 60, 70 días, y 7.5, 10, 12.5% por ejemplo
- hipótesis para JPM: personalizar pronóstico de adelgazamiento según estado emocional basal
- apoyo estadístico: 50€ por hora y media, 1-2 veces al mes. Vendría a través de ISCYL, en bonos, probable que para mí, y lo a Árpi
- no abusar de chatgpt! comprobar siempre las propuestas. 


[12.25] haladunk, egyre több metadata egyre koncentráltabban van jelen a bázisban. kövi nagy lépés a 10% eventek időpontjának megállapítása. utána illeszthetőek a regressziós modellek. nem hiszem, h ez ma meglesz, ezzel együtt haladunk ahogy kell. 

[7.40] Mára miket kéne csinálni? 
- exploratory association plot with 60-day deltas and emotional scores; picking first records from multirec patients
- possibly: making a **larger wide df**, that contains the most data possible from patients
- regarding survival analysis: 
    - define input data and its structure
    - exclude inmediate dropouts
    - EDA: descriptive stats and recommended KM curves
    - advanced models: 
        - define train/test sets
        - fit simple Cox model, 1 or a few variables
        - fit competing risk models with time strata (because of proportional hazards assumption, although maybe that is for Cox actually?)
        - maybe fit parametric models

### 14/4/25

[AI GENERATED ANALYSIS PROTOCOL BASED ON DISCUSSIONS THROUGHOUT THE DAY]

**Analysis Protocol: Emotional Behaviors and Weight Loss Success**  
**Objective**: Understand how baseline emotional/eating behaviors predict the time it takes to achieve 10% weight loss, while addressing dropouts and statistical pitfalls.

**1. Data Preparation**  
**Goal**: Clean and structure your data for survival analysis.  
- **Define Events**:  
  - **Success**: The day a participant first reaches ≥10% weight loss (e.g., Day 45).  
  - **Censoring**: The last day a participant was measured if they dropped out or never reached 10% loss.  
- **Exclude Immediate Dropouts**:  
  - **Why?** Participants with only 1 measurement (e.g., Day 0) provide no information about *time-to-event*. Including them would artificially inflate the number of "censored" cases without meaningful follow-up.  
  - **How?** Remove anyone with only baseline data (no follow-up).  

 **2. Exploratory Analysis**  
**Goal**: Understand patterns in your data before modeling.  
- **Descriptive Stats**:  
  - Report averages for continuous variables (e.g., "Average emotional eating score: 6.2/10").  
  - Report percentages for categorical variables (e.g., "60% female").  
- **Kaplan-Meier Curves**:  
  - **What?** A graph showing the probability of *not* achieving 10% weight loss over time.  
  - **How?** Compare groups (e.g., high vs. low emotional eating). If the curves separate, the variable might matter.  

**3. Primary Analysis: Cox Regression**  
**Goal**: Test if emotional behaviors predict *how quickly* people lose weight.  
- **Model 1 (Simple)**:  
  - Predictor: One emotional variable (e.g., emotional eating score).  
  - Output: **Hazard ratio (HR)**.  
    - HR > 1: Faster weight loss.  
    - HR  0.5**: Model is better than random guessing.  

**5. Sensitivity Analyses**  
**Goal**: Check if results hold under different assumptions.  
- **Competing Risks**:  
  - **What?** Treat dropout as a competing event (e.g., someone quits and can’t lose weight).  
  - **How?** Use Fine-Gray models instead of Cox.  
- **Parametric Models**:  
  - **What?** Assume the risk of weight loss decreases over time (e.g., Weibull model).  
  - **Why?** Compare results to Cox for consistency.  

**6. Reporting Results**  
**Tables**:  
- Show hazard ratios (HRs), 95% confidence intervals (CIs), and *p*-values for all models.  
**Figures**:  
- Kaplan-Meier curves for key predictors.  
- Forest plots comparing HRs across subgroups.  

**Key Questions to Answer**  
1. Does emotional eating slow down or speed up weight loss?  
2. Do effects change over time (e.g., matter more in the first month)?  
3. Are results robust after adjusting for age, sex, and BMI?  

[jotting down paper notes taken over the day] Egyértelmű, h cikkre kell fókuszálni, az is, h ehhez a Cox reg fontos concept. Ott kell felvenni a fonalat, ahol letettem - mérések és metaadatok linkelése. Ha azt jól csináltam, ott meg is vannak a 60 napos delták - igaz még az input adatokat tisztítani kell, pl duplikáltak kidobása stb, visszamenőleg tisztítani kell a pipelinet. Illetve, ezek tentatív elemzések és nem végleges adatok lesznek, egészen addig amíg a Measurements átnézése meg nem történik. 

[JOTS ON COX/SURVIVAL ANALYSIS RESEARCH] 
- Cox model: effect of a unit increase in a covariate: multiplicative
- low risk vs high risk / low/high emotionality
- hazard ratio: the key concept, that's estimated in a Cox model. Associated with a risk factor or predictor for a given endpoint - meaning? Its prerequisite is assesment of proportional hazards. Meaning? 
- Kaplan-Meier: simplest SA, after comes Cox, penalized Cox models like ridge, lasso, elastic net, and more robust MLs for SA
- SA: effects of several factors on an event happening - rate of this event happening is the HAZARDS RATE; risk of event X at time t
- want to find out the influence of factors A, B, C, D, X, Y, Z on the RATE OF THE EVENT HAPPENING? 
- covariates: predictor variables, factors
- linking COVARIATES data to TIME OF EVENT data - establish LINKS with SA
- selecting best predictive variable is important    
- concordance index: correlation between predicted risk scores and time point
- Cox model needs train/test data too!
- Penalized models, Random survival forest, survival SMVs, survival GB can have interesting advantages, 
- scikit-survival: https://scikit-survival.readthedocs.io/en/stable/user_guide/index.html

[BRAINSTORM: HOW TO USE IT]
- this is like a regression - but parts of the training data is CENSORED due to dropout, death, and in some cases, the outcome is just not achieved - > we can use data from ALL patients and fitting and adequate survival model we can account for dropouts
- existing conditions, comorbidities as confounders should also be accounted for; and a subset with genetics, also maybe look for links in weight gain reasons
- observe 60 days - some drop out, some lose 10%, some don't
- adjust for age, sex, baseline BMI, comorbidities etc
- Still, SA focuses on **time to event happening**, which may not be the most revealing test to look for associations between emotional eating and WL outcomes. 
- clause 0, if your supervisor asks you to look into something, you do
- clause 1, survival analysis looks good for the case where we have dropouts, successes, different durations till success, no success and all
- what can be complicated is the relatively large nr of covariates in our analyses. we need to adjust for demographics, anthropometrics, emotional and clinical variables. maybe more complex models are the answer here. scikit-surv guide mentions versatility of SSVMs for example. Perplexity recommends those for complex, highD, nonlinear datasets. 
- what we *really care about* is not the time to success, but the *influence of certain covariates on success*. 
- **competing risk models**, like Fine-Gray, accounts for meaningful dropouts - like, if emotional values influence on dropout, then dropout should be accounted for as a competing risk that prevents the outcome of interest. If a covariate predicts higher risk for a competing risk, that is insightful. Otherwise, non-informative dropouts should be censored, and the researcher should stick to a normal Cox model. 
- regarding the analysis: time-to-event analysis should be the best, like identifying the time needed to achieve a 10% WL. OK, that implies doing the absolute to relative calculation maybe more times, but I think that can be a dynamic part of the code, no need to calculate for every measurement, only pulling a function on measurements of interest. 
- IMPORTANT: Proportional hazards is some kind of assumption in Cox, that the effect size of one covariate is constant over time, does not change. Like if hunger influences weight this much at baseline, the size of the influence will stay constant. **This is risky** and needs to be adjusted for - emotional values can change, and their effects can change. Some kind of test needs to be ran to check whether this assumption is violated (Schoenfeld residuals, 'check_assumptions' in scikit-survival), and if it is, stratifying the data for periodic blocks and doing different analyses for each block can be a way to go. Like, fit a new model for every 20 days, and engineer it well enough to link the different ones. In every model, the covariates are baseline (cross-sectional basically), the outcome is derived from the measurements that are continuously recorded. The goal is to see how the weight decreases, in function of some variables, and how long it takes to reach 5, 10, 15, 20% WL. 

[QUESTIONS]
- what should the input data structure look like for the survival models? 



### 5/4/25

[**problems for future self**] még mindig messy a notebook, ezt a pipelinet nem is látom át egészen, és duplikált mérések vannak az inputban, mert colclean, nem rowclean inputtal fut le a kód

[09.11] Csökkenni fog a hosszútávú adatokkal rendelkező páciensek száma az érzelmi datasetben, ha 60 nap followupra szűrünk, többen dropoutoltak. Viszont a baseline adatokban az érzelem - baseline obesity és a droput rate közötti összefüggéseket is meg lehet nézni

makin progress! két jó ötlet, és megvan a 2 df a 60day delta weight értékekkel. 

### 4/4/25

[16.25]
Szeretnék mock analíziseket csinálni, amelyekben összehasonlítom az eating behavior és a weight loss outcome adatokat. Mivel egy ember több kezelésen megy át, és a kezelések hossza változó, a tudományos pontosság miatt fontos, hogy fix vagy nagyjából fix időtartamú kezeléseket nézzünk (persze később teljes kezelésre is rá lehet vetíteni, 60-day weight loss vs overall treatment outcome). 

Ahhoz, hogy a fix időtartamú követési időszakok első és utolsó mérési adatát megszerezzük, ahhoz minden méréshez hozzá kell rendelni a hozzá tartozó kezelést, illetve receptet, amiből megtudjuk, hogy a kezelés melyik szakaszában van a páciens, és hogy az adott szakasz meddig tartott, illetve hogy maga az egész kezelés milyen hosszú volt. Most ami megvan, azt hiszem, hogy a kezelések és receptek hozzá vannak rendelve az egyes mérésekhez. 

Ezek a hozzárendelések long formájúak, és tovább vannak fejlesztve egy olyan táblázatban, ahol minden prescription, step és medical record első és utolsó mérése van, illetve a köztük eltelt idő és súlyváltozás. Ez egyfajta megközelítés, olvashatóbb, viszont elvesznek a köztes adatok. Idáig jutottam, és itt megakadtam egy kicsit, hogy akkor most hogy is tovább? Nem is vagyok biztos benne, hogy egyáltalán látom a problémát, ami megakasztott. 

Amit szeretnék, az 1 treatment cikluson belüli, 1 medical recordhoz tartozó 2 hónap körüli mérések súlyváltozása. Az kulcsfontosságú, hogy egy medical recordhoz tartozzanak, az viszont, hogy melyik lépéshez tartoznak, az inkább kiegészítő információ. Sok kontextust ad, mert tudjuk, hogy keto vagy nem keto fázisban van-e a páciens, de nem végzetes hiba, ha ez az infó elvész. Ami végzetes hiba lenne, az pl. két kezelési ciklus adatainak az összevonása. 

Tehát a most meglévő megközelítéssel, a prescription/step/record szinten is első és utolsó adatot tartalmazó táblával az a baj, hogy túl bonyolult, és talán adatot is veszítek. Elég lenne egy olyan tábla, ahol 1 medical recordhoz tartozó adatok vannak, baseline és 3, +/- 1 hónap cutoffnál. Tehát a most (piszkozat állapotban) 'linked' nevű táblázat sorait kéne patient>medical record szerint csoportosítani, measurement_date szerint rendezni, és az első méréstől számított 2-4 hónap közötti, 3-hoz legközelebbi értéket kinyerni, ezzel számolni egy pivoted delta weight táblázatot, amiben benne van a step is, mint metadata. Ezeket a súlyadatokat lehet már korreláltatni az érzelmi adatokkal, és a megfelelő metadata ismeretében könnyebb a cleaning és az értelmezés is. 

